# Language Modeling using Ngram

In this Exercise, you are going to use NLTK which is a natural language processing library for python to create a bigram language model and its variation. You will build one model for each of the following type and calculate their perplexity:
- Unigram Model
- Bigram Model
- Bigram Model with add one estimation
- Bigram Model with Interpolation
- Bigram Model with Kneser-ney Interpolation
- Neural LM



In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# #download corpus
# import shutil
# shutil.copy("/content/drive/MyDrive/FRA 501 IntroNLP&DL/Dataset/BEST2010.zip", "/content/BEST2010.zip")
# !unzip BEST2010.zip

In [3]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually use as a standard benchmark for various NLP tasks includeing language modeling. BEST2010 is separated into 4 domain article, encyclopedia, news and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [4]:
# We choose news domain as our dataset
best2010=[]
fp= io.open('Dataset/BEST2010/news.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    best2010.append(line.strip()[:-1])
fp.close()
all_vocabulary =set()
total_word_count =0
for line in best2010:
    for word in line.split('|'):        
        all_vocabulary.add(word)
        total_word_count+=1

In [5]:
#For simplicity, we assumes that each line is a sentence.
print ('Total sentences in BEST2010 news dataset :\t'+ str(len(best2010)))
print ('Total word counts in BEST2010 news dataset :\t'+ str(total_word_count))
print ('Total vocabulary in BEST2010 news dataset :\t'+ str(len(all_vocabulary)))

Total sentences in BEST2010 news dataset :	30969
Total word counts in BEST2010 news dataset :	1660190
Total vocabulary in BEST2010 news dataset :	35488


We separate out input into 2 sets, train and test data with 70:30 ratio

In [6]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary =set()
train_word_count =0
for line in train:
    for word in line.split('|'):        
        train_vocabulary.add(word)
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))
print ('Total vocabuary in BEST2010 news training dataset :\t'+ str(len(train_vocabulary)))
# We will use 1/vocab_size as a default value for unknown word
unk_value = math.pow(len(train_vocabulary),-1)

Total sentences in BEST2010 news training dataset :	21678
Total word counts in BEST2010 news training dataset :	1042797
Total vocabuary in BEST2010 news training dataset :	26240


# Unigram

In this section, we will demonstrate how to build a unigram language model <br>
**Important note:** <br>
**\<s\>** = sentence start symbol <br>
**\</s\>** = sentence end symbol 

In [7]:
def getUnigramModel(data):
    model = defaultdict(lambda: 0)
    word_count =0
    for sentence in data:
        sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
        for w1 in sentence.split('|'):
            model[w1] +=1.0
            word_count+=1
    for w1 in model:
        model[w1] = model[w1]/(word_count)
    return model

In [8]:
model = getUnigramModel(train)

In [9]:
def getLnValue(x):
    if x >0.0:
        return math.log(x)
    else:
        return math.log(unk_value)

In [10]:
#problability of 'นายก'
print(getLnValue(model[u'นายก']))
#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(model[u'นายกรัฐมนตรี']))
#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(model[u'นายก'])+getLnValue(model[u'ได้'])+ getLnValue(model[u'ให้'])+getLnValue(model[u'สัมภาษณ์'])+getLnValue(model[u'กับ'])+getLnValue(model[u'สื่อ'])+getLnValue(model['</s>'])
print ('Problability of a sentence', math.exp(prob))

-6.551526663995246
-10.175040243058024
Problability of a sentence 5.617210748667918e-18


## TODO #1 **Calculate perplexity**

In order to compare language model we need to calculate perplexity. In this task you should write a perplexity calculation code for the unigram model. The result perplexity should be around 556.39 and
476.07 on train and test data.

In [11]:
def calculate_sentence_ln_prob(sentence, model):
    # word = sentence.spilt('|')
    # ln_prob = 0
    # for ไล่เเต่ละคำใน sentence เพื่อคำนวณ LnValue --> sum ln_prob ทุกคำ
    # return ln_prob
    pass

def perplexity(test,model):
    # ln_prob = 0
    # word_count = 0
    # for ไล่เเต่ละ sentence --> คำนวณ calculate_sentence_ln_prob ของเเต่ละ sentence --> sum ln_prob ทุก sentence
    # return exp(-ln_prob/word_count)
    pass

In [12]:
print(perplexity(train,model))
print(perplexity(test,model))

None
None


# Bigram

Next, you will create a better language model than a unigram (which is not much to compare with). But first, it is very tedious to count every pair of words that occur in our corpus by ourselves. In this case, nltk provide us a simple library which will do it for us.

In [13]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'

print('This is how nltk generate bigram.')
for w1,w2 in bigrams(sentence.split(), pad_right=True, pad_left=True):
    print (w1,w2)
print('None is used as a start and end of sentence symbol.')

This is how nltk generate bigram.
None I
I always
always search
search google
google for
for an
an answer
answer .
. None
None is used as a start and end of sentence symbol.


Now, you should be able to implement a bigram model by yourself. Also, you must create a new perplexity calculation for bigram. The result perplexity should be around 58.78 and 146.26 on train and test data.

## TODO #2 **Create a Bigram Model**

In [14]:
def getBigramModel(data):
    ###FILL YOUR CODE HERE###
    # unigram_count = defaultdict(lambda: 0.0)
    # bigram_count = defaultdict(lambda: 0.0)

    # for เเต่ละ sentence
    #   for เเต่ละ token ในรูปเเบบ bigram ที่ generate ขึ้นมา
    #     bigram_count[?] = ?
    #     unigram_count[?] = ?

    # for ไล่เเต่ละ token ใน bigram ทั้งหมด
    #   model[?] = ?
    return model

model = getBigramModel(train)

## TODO #3 **Calculate Perplexity for Bigram Model**



In [15]:
def calculate_sentence_ln_prob(sentence, model):
    # คำนวณจาก getBigramModel อย่าเติม <s> หรือ </s> เอง !!!
    pass

def perplexity(test,model):
    pass

In [16]:
print (perplexity(train,model) )
print (perplexity(test, model))

# 58.78942889767147
# 146.26539331038614

None
None


# Smoothing

Usually any ngram models have a sparsity problem, which means it does not have every possible ngram of words in the dataset. Smoothing techniques can alleviate this problem. In this section, you will implement two basic smoothing methods laplace smoothing and interpolation for bigram.

## TODO #4 **Bigram with add-one estimation**

In [17]:
#Laplace Smoothing
def getBigramWithAddOneEstimation(data):
    #Fill code here
    return model

model = getBigramWithAddOneEstimation(train)
print (perplexity(train,model) )
print (perplexity(test, model))

# 974.8134581679766
# 1098.1622194979489

None
None


## TODO #5 **Bigram with Interpolation**
lambda value is 0.7 for bigram, 0.25 for unigram, and 0.05 for unknown word

In [18]:
#interpolation
def getBigramWithInterpolation(data):
    #Fill code here
    # unigram_count = defaultdict(lambda: 0.0)
    # bigram_count = defaultdict(lambda: 0.0)
    # model = defaultdict(lambda: 0.0)

    # for เเต่ละ sentence
    #   for เเต่ละ token ใน bigram ที่ generate ขึ้นมา
    #     bigram_count[?] = ?
    #     unigram_count[?] = ?


    # for เเต่ละ key ใน bigrams
    #   bigram_prob
    #   unigram_prob
    #   model[key] = สูตร bigram, unigram, unk_value (1/vocab)
    return model
    
model = getBigramWithInterpolation(train)
print (perplexity(train,model))        
print (perplexity(test,model))

# 73.38409869825665
# 172.67485908813356

None
None


# Language modeling on multiple domains

Sometimes, we do not have enough data to create a language model for a new domain. In that case, we can improvised by combining several models to improve result on the new domain.

In this exercise you will try to merge two language models from news and article domains to create a language model for the encyclopedia domain.

In [19]:
# create article data
encyclo_data=[]
fp= io.open('Dataset/BEST2010/encyclopedia.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    encyclo_data.append(line.strip()[:-1])
fp.close()

First, you should try to calculate perplexity of your bigram with interpolation using "news data" (train) on "encyclopedia data" (test). The result perplexity should be around 727.35.

For your information, a bigram model with interpolation using "ariticle data" (train) to test on "encyclopedia data" (test) has a perplexity of 505.79.

In [20]:
# print perplexity of bigram with interpolation on article data        
# 727.3502637212223
print (perplexity(encyclo_data,model))

None


## TODO #6 
Write a model that produce 450.0 or less perplexity on encyclopedia data without using data from the encyclopedia as training data. (Hint : Try to combine a model with news data and a model with article data together.)

In [21]:
# Fill code here
# 428.85251789073953 (on combined data)
print('Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data',perplexity(encyclo_data, combined_model))

NameError: name 'combined_model' is not defined

## TODO #7 
## Kneser-ney on "News"

<!-- Reimplement equation 4.33 in SLP textbook (https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf) -->

Implement Bigram Knerser-ney LM. The result perplexity should be around 71.14054002208687 and 174.02464248000433 on train and test data. 


In [ ]:
# Fill codehere

#-------------------------------------------
# Create unigram and bigram counting table
#-------------------------------------------
# unigram_count = defaultdict(lambda: 0.0)
# bigram_count = defaultdict(lambda: 0.0)
# model = defaultdict(lambda: 0.0)

print (perplexity(train,model))        
print (perplexity(test,model))

# 71.14054002208687
# 174.02464248000433

## TODO #8
## Neural LM 
do it on news corpus that we splitted into train and test sets at the beginning of this exercise. 

In [ ]:
#find the perplexity of the model
#there are many ways to do this. e.g.:
#https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/